# COVID-19 Detection on CT Scans

In this notebook, we will optimize a pre-trained models for COVID-19 detection and inferece using OpenVINO.

The dataset and models are from https://github.com/kaushikjadhav01/COVID-19-Detection-Flask-App-based-on-Chest-X-rays-and-CT-Scans/

## 1. Import Libraries

In [1]:
import os
import sys
import time
import zipfile
from glob import glob
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import mo_tf
import numpy as np
import tensorflow as tf
from openvino.inference_engine import IECore
from tqdm import tqdm

## 2. Download dataset and models
Download and unzip file if it does not exist

In [2]:
if not os.path.exists('data_and_models'):
    !(wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1dA-rdmDmCGa3xxW5KpfLJdo7M54lPcQq' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1dA-rdmDmCGa3xxW5KpfLJdo7M54lPcQq" -O data_and_models.zip && rm -rf /tmp/cookies.txt)
    with zipfile.ZipFile('data_and_models.zip', 'r') as zf:
        zf.extractall()
    os.rename('data and models', 'data_and_models')
    !(rm data_and_models.zip)
print('Dataset and models are ready.')

Dataset and models are ready.


## 3. Optimize Model
OpenVINO model optimizer does not support Keras H5 models. We need to convert Keras H5 models into TensorFlow SavedModel.

Select the model you want to convert, here we use `resnet_chest`.
Model names:

- resnet_chest
- inceptionv3_chest
- vgg_chest
- xception_chest

In [3]:
model_name = 'resnet_chest'
# model_name = 'inceptionv3_chest'
# model_name = 'vgg_chest'
# model_name = 'xception_chest'

if not os.path.exists(model_name):
    model_h5 = tf.keras.models.load_model(f'data_and_models/models/{model_name}.h5')
    tf.saved_model.save(model_h5, model_name)

#### Prepare commnad for optimizing the model
Construct the command for model optimizer. We save the converted model in the directory named after selected model.

In [4]:
# Get the path to the Model Optimizer script
mo_path = str(Path(mo_tf.__file__))
ir_path = Path(os.path.join(model_name, 'saved_model.xml'))

mo_command = f'''"{sys.executable}"
                 "{mo_path}" 
                 --saved_model_dir "{model_name}" 
                 --input_shape "[1,224,224,3]"
                 --data_type FP32
                 --output_dir "{model_name}" 
                 '''
mo_command = ' '.join(mo_command.split())
print('Model Optimizer command to convert TensorFlow to OpenVINO:')
print(mo_command)

Model Optimizer command to convert TensorFlow to OpenVINO:
"/opt/conda/bin/python" "/opt/conda/lib/python3.7/site-packages/mo_tf.py" --saved_model_dir "resnet_chest" --input_shape "[1,224,224,3]" --data_type FP32 --output_dir "resnet_chest"


#### Run Model Optimizer if the IR model file does not exist

In [5]:
if not ir_path.exists():
    print("Exporting TensorFlow model to IR... This may take a few minutes.")
    ! $mo_command
else:
    print(f"IR model {ir_path} already exists.")

IR model resnet_chest/saved_model.xml already exists.


## 4. Verify Optimized Model

#### Load Original Model for comparison

In [6]:
model_h5 = tf.keras.models.load_model(f'data_and_models/models/{model_name}.h5')

def infer_h5(image_path):
    image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    resized_image = cv2.resize(image, (224, 224)) / 255
    X = np.expand_dims(resized_image, axis=0)
    start_time = time.time()
    result = model_h5.predict(X)
    t = time.time() - start_time
    return result, t

# warm up
infer_h5('data_and_models/data/chest/Chest_COVID/ryct.2020003.fig2-a.png')

(array([[0.9978103 , 0.00218971]], dtype=float32), 1.4140608310699463)

#### Load Optimized Model

In [7]:
ie = IECore()
net = ie.read_network(f'{model_name}/saved_model.xml', f'{model_name}/saved_model.bin')
exec_net = ie.load_network(net, "CPU")
input_key = list(exec_net.input_info)[0]
output_key = list(exec_net.outputs.keys())[0]
network_input_shape = exec_net.input_info[input_key].tensor_desc.dims

def infer_ir(image_path):
    image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    resized_image = cv2.resize(image, (224, 224)) / 255
    input_image = np.expand_dims(np.transpose(resized_image, (2, 0, 1)), 0)
    start_time = time.time()
    result = exec_net.infer(inputs={input_key: input_image})[output_key]
    t = time.time() - start_time
    return result, t

#### Define Method for Comparing Orginal and Optimized Models

In [8]:
def comp_model(image_path):
    result_h5, time_h5 = infer_h5(image_path)
    result_ir, time_ir = infer_ir(image_path)
    
    print('Result:')
    print('Keras H5:\t', result_h5[0])
    print('OpenVINO:\t', result_ir[0])

    print('Time:')
    print(f'Keras H5:\t {time_h5:.3f} sec')
    print(f'OpenVINO:\t {time_ir:.3f} sec')
    
    print(f'Speedup:\t x{time_h5/time_ir:.3f}')

#### COVID-19 Positive Sample

In [9]:
image_path = 'data_and_models/data/chest/Chest_COVID/ryct.2020003.fig2-a.png'
comp_model(image_path)

Result:
Keras H5:	 [0.9978103  0.00218971]
OpenVINO:	 [0.99780995 0.00219002]
Time:
Keras H5:	 0.418 sec
OpenVINO:	 0.190 sec
Speedup:	 x2.201


#### COVID-19 Negative Sample

In [10]:
image_path = 'data_and_models/data/chest/Chest_NonCOVID/0a4d9634-7ee8-4512-ba83-6ff5e352b2c2.jpg'
comp_model(image_path)

Result:
Keras H5:	 [0.07289597 0.92710406]
OpenVINO:	 [0.07287621 0.92712384]
Time:
Keras H5:	 0.393 sec
OpenVINO:	 0.185 sec
Speedup:	 x2.123


## 5. Copy to PHFS for Deployment

To deploy the optimized models, we need to put it in PHFS first.

In [11]:
phfs_dir = f'/phfs/openvino/{model_name}'
if not os.path.exists(phfs_dir):
    os.makedirs(phfs_dir)

xml_file = f'{model_name}/saved_model.xml'
bin_file = f'{model_name}/saved_model.bin'
!cp $xml_file $phfs_dir
!cp $bin_file $phfs_dir

## Appendix: Classification Report

In [12]:
covid_files = glob('data_and_models/data/chest/Chest_COVID/*')
noncovid_files = glob('data_and_models/data/chest/Chest_NonCOVID/*')

print('Infering COVID images:')
covid_pred = [infer_ir(f)[0] for f in tqdm(covid_files, position=0, leave=True)]
print('Infering Non-COVID images:')
noncovid_pred = [infer_ir(f)[0] for f in tqdm(noncovid_files, position=0, leave=True)]

  0%|          | 0/435 [00:00<?, ?it/s]

Infering COVID images:


  0%|          | 1/505 [00:00<01:02,  8.10it/s]

Infering Non-COVID images:


100%|██████████| 505/505 [01:18<00:00,  6.78it/s]


In [13]:
covid_pred_bin = [np.argmax(x) for x in covid_pred]
noncovid_pred_bin = [np.argmax(x) for x in noncovid_pred]

y_pred_bin = covid_pred_bin
y_pred_bin.extend(noncovid_pred_bin)
y_test_bin = [0] * len(covid_files)
y_test_bin.extend([1] * len(noncovid_files))

from sklearn.metrics import classification_report
print(classification_report(y_test_bin, y_pred_bin))

              precision    recall  f1-score   support

           0       0.72      0.91      0.80       435
           1       0.90      0.69      0.78       505

    accuracy                           0.79       940
   macro avg       0.81      0.80      0.79       940
weighted avg       0.82      0.79      0.79       940

